<a href="https://colab.research.google.com/github/ahforoughi/Knowledge-based-recomender-system/blob/main/KB_recomender_system_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from ast import literal_eval



In [181]:
data_path = "/content/IMDb movies.csv"
data = pd.read_csv(data_path)
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [182]:
print("data shape before shrink", data.shape)
data = data.iloc[:1000, : ]
data = data[['title', 'year', 'date_published', 'genre', 'duration', 'country', 'avg_vote', 'votes']]
print("data shape after shrink", data.shape)


data shape before shrink (85855, 22)
data shape after shrink (1000, 8)


In [183]:
data.dtypes
data.head()

,title,year,date_published,genre,duration,country,avg_vote,votes
0,Miss Jerry,1894,1894-10-09,Romance,45,USA,5.9,154
1,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,6.1,589
2,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",5.8,188
3,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,5.2,446
4,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,7.0,2237


In [184]:
def convert_int(x):
    try:
        return int(x)
    except: 
        return 0
data['year'] = data['year'].apply(convert_int)
data.dtypes

title              object
year                int64
date_published     object
genre              object
duration            int64
country            object
avg_vote          float64
votes               int64
dtype: object

In [185]:
#Convert all NaN into stringified empty lists
data['genre'] = data['genre'].fillna('')

In [186]:
data['genre'] = data['genre'].str.split(',')
data = data.explode('genre')

In [187]:
# check null values for each column
columns = list(data.columns)
for i in columns:
    print (f"null values for {i} =", data[i].isnull().sum())


null values for title = 0
null values for year = 0
null values for date_published = 0
null values for genre = 0
null values for duration = 0
null values for country = 0
null values for avg_vote = 0
null values for votes = 0


In [188]:
movies = data.copy()
movies.dtypes
low_time = 30
test = movies[movies['duration']>=low_time]
test

,title,year,date_published,genre,duration,country,avg_vote,votes
0,Miss Jerry,1894,1894-10-09,Romance,45,USA,5.9,154
1,The Story of the Kelly Gang,1906,1906-12-26,Biography,70,Australia,6.1,589
1,The Story of the Kelly Gang,1906,1906-12-26,Crime,70,Australia,6.1,589
1,The Story of the Kelly Gang,1906,1906-12-26,Drama,70,Australia,6.1,589
2,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",5.8,188
...,...,...,...,...,...,...,...,...
998,Top Speed,1930,1930-08-24,Musical,80,USA,6.1,172
998,Top Speed,1930,1930-08-24,Romance,80,USA,6.1,172
999,True to the Navy,1930,1930-05-25,Action,71,USA,6.7,103
999,True to the Navy,1930,1930-05-25,Comedy,71,USA,6.7,103


In [210]:
 def weighted_rating(x, C, m):
     v = x['votes']
     R = x['avg_vote']
     ## Compute the weighted score
     return (v/(v+m) * R) + (m/(m+v) * C)


 def search_in_data(genre, low_time, high_time, low_year, high_year, movies, percent=0.8):

    result = movies[(movies['genre']==genre) &
                    (movies['duration']>=low_time) &
                    (movies['duration']<=high_time) &
                    (movies['year']>=low_year) & 
                    (movies['year']<=high_year)
    ]

    vote_ave = result['avg_vote'].mean()
    vote_thresh = result['votes'].quantile(percent)

    f_results = result.copy().loc[(result['votes'] > vote_thresh)]


    f_results['score'] = f_results.apply(weighted_rating, C=vote_ave, m=vote_thresh, axis=1)
    f_results = f_results.sort_values('score', ascending=False) 

    return f_results



In [214]:
# Ask for input
# genre = input("Enter your prefered genre: ")
# low_time = int(input("Enter your prefered low_time: "))
# high_time = int(input("Enter your prefered high_time: "))
# low_year = int(input("Enter your prefered low_year: "))
# high_year = int(input("Enter your prefered high_year: "))


# test data
genre = 'Crime'
low_time = 40
high_time = 90
low_year = 1920
high_year = 1930

KBS = search_in_data(genre, low_time, high_time, low_year, high_year, data)#.head()
KBS



,title,year,date_published,genre,duration,country,avg_vote,votes,score
632,I dannati dell'oceano,1928,1928-09-29,Crime,76,USA,7.5,3860,7.240160
610,Le notti di Chicago,1927,1927-10-29,Crime,80,USA,7.6,2545,7.217102
499,Il pensionante,1927,1927-02-14,Crime,68,UK,7.3,10024,7.204358
707,Ricatto,1929,1929-10-06,Crime,85,UK,7.0,9412,6.933694
702,Asfalto,1929,1929-03-12,Crime,90,Germany,7.4,1328,6.931817
455,Il trio infernale,1925,1927,Crime,86,USA,7.2,1613,6.865428
851,Carcere,1930,1930-06-14,Crime,87,USA,7.1,1849,6.831092
266,Il castello di Vogelod,1921,1921-04-07,Crime,75,Germany,6.2,1391,6.307738
